In [19]:
import psycopg2 as pg
import pandas as pd

In [16]:
# Credentials
host ='comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com' 
port = 5432
database ='comunidadedsdb'
user ='member'
pwd ='cdspa'

In [25]:
# Conn creation
conn = pg.connect(host=host, 
                   port=port,
                   database=database,
                   user=user, 
                   password=pwd)

## 1.0 Schema Query

In [8]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""
cursor.execute( query_schema)

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()


## 2.0 Tables Query

In [14]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'
"""
cursor.execute( query_tables)

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()


[('users',), ('vehicle',), ('insurance',)]


## 3.0 Collect Data using Psycopg2

In [18]:
cursor = conn.cursor()

query_table_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""
cursor.execute( query_table_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()


[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [20]:
df = pd.DataFrame( record )
df.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


## 4.0 Collect Data using Pandas

In [21]:
import pandas.io.sql as psql

In [26]:
query_table_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle   v  ON ( u.id = v.id )
                       LEFT JOIN pa004.insurance i ON ( u.id = i.id  )
    WHERE u.age > 44
   """

df = psql.read_sql( query_table_users, conn )
df.head()


,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,8,Female,56,28.0,26.0,8,1,1-2 Year,Yes,8,0,32031.0,72,1
1,11,Female,47,35.0,124.0,11,1,1-2 Year,Yes,11,0,47576.0,46,1
2,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0
3,42,Female,54,28.0,122.0,42,1,1-2 Year,No,42,0,38560.0,184,0
4,53,Female,49,3.0,152.0,53,1,1-2 Year,No,53,1,25553.0,130,0


In [29]:
df1 = pd.read_sql( query_table_users, conn )
df1 = df1.drop( df1.columns[[5, 9]], axis=1)
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0
